In [ ]:
"""
Created on Tue Jun 21 14:55 2022

This script uses the CV parameters from "group_CV_parameters.ipynb" to compute the integrated melt and 
melt near the grounding line on the synthetically independent evaluation dataset
+
computes evaluation metrics using original parameters

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  
from tqdm.notebook import trange, tqdm
import time
import assess_param_funcs.melt_functions as meltf
import assess_param_funcs.useful_functions as uf
from assess_param_funcs.constants import *
import os, glob

In [ ]:
home_path='/bettik/burgardc/'

READ IN DATA

In [ ]:
nemo_run = 'OPM021'

In [ ]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'
plot_path = '/bettik/burgardc/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
file_TS = file_TS_orig.sel(Nisf=large_isf).sel(profile_domain=[50,1000])

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
# creates stack mask to assign indexes to given ice shelves to avoid as much including large NaN-fields in 
# calculations and reduce memory usage this way
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

# identify location of the 1st box in the 5-box setup which is our region "near the grounding line"
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

SIMPLE

In [ ]:
# choose CV_type
CV_type = 'shelves' # can be 'shelves' or 'time'

In [ ]:
inputpath_chunk_info = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/'
info_file = pd.read_csv(inputpath_chunk_info+'info_chunks.txt',header=None, index_col=0)

In [ ]:
# read gammas as grouped in "group_CV_parameters.ipynb"
gammas_CV_simple = xr.open_dataset(outputpath_simple_all+'gammas_simple_CV_'+CV_type+'.nc')

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 

nisf_list = geometry_info_1D.Nisf

for mparam in simple_param_types:
    
    if CV_type == 'shelves':

        
        eval_1D_list = []
        for kisf in tqdm(nisf_list):

            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_simple['slope'].sel(param=mparam).sel(isf=kisf).drop('isf')
        
            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf([kisf], T_S_profile, 
                                                                    geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                    mparam, gamma, U_param=True, 
                                                                    box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
            eval_1D_list.append(eval_1D)
            
        eval_1D_CV_isf = xr.concat(eval_1D_list, dim='Nisf')
            
        eval_1D_CV_isf.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        
        eval_1D_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_simple['slope'].sel(param=mparam).sel(tblock=tblock).drop('tblock')
                eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                            geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                            mparam, gamma, U_param=True, 
                                                                            box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D)
                eval_1D_list.append(eval_1D)
            
        eval_1D_CV_time = xr.concat(eval_1D_list, dim='time')
    
        eval_1D_CV_time.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVtime.nc')



PLUMES

In [ ]:
# choose CV_type
CV_type = 'shelves' # can be 'shelves' or 'time'

In [ ]:
plume_param_types = ['lazero19_2','lazero19_modif2'] 

nisf_list = geometry_info_1D.Nisf

for mparam in plume_param_types:
    
    if CV_type == 'shelves':
        
        gammas_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_gamma_CV'+CV_type+'.nc')
        E0s_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_E0_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_plumes['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            E0 = E0s_CV_plumes['E0'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
        
            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf([kisf], T_S_profile, 
                                                                 geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                 mparam, gamma, E0=E0, 
                                                                 box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
            eval_1D_list.append(eval_1D)
            
        eval_1D_CV_isf = xr.concat(eval_1D_list, dim='Nisf')
            
        eval_1D_CV_isf.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        gammas_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_gamma_CV'+CV_type+'.nc')
        E0s_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_E0_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_plumes['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock')
                E0 = E0s_CV_plumes['E0'].sel(param=mparam).sel(tblock=tblock).drop('tblock')

                eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, E0=E0, 
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)
                eval_1D_list.append(eval_1D)
            
        eval_1D_CV_time = xr.concat(eval_1D_list, dim='time')
    
        eval_1D_CV_time.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVtime.nc')



BOXES

In [ ]:
# choose CV_type
CV_type = 'shelves' # can be 'shelves' or 'time'

In [ ]:
nisf_list = geometry_info_1D.Nisf

for nD_config in range(1,5):
    
    pism_version = 'yes'
    picop_opt = 'no'
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
            
    if CV_type == 'shelves':
        
        gammas_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_gamma_CV'+CV_type+'.nc')
        Cs_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_C_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_boxes['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf') * 10**(-5)
            C = Cs_CV_boxes['C'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf') * 10**6
        
            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf([kisf], T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, C=C, angle_option='appenB',
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                     box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                                     verbose=False)
            eval_1D_list.append(eval_1D)
            
        eval_1D_CV_isf = xr.concat(eval_1D_list, dim='Nisf')
            
        eval_1D_CV_isf.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        gammas_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_gamma_CV'+CV_type+'.nc')
        Cs_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_C_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_boxes['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock') * 10**(-5)
                C = Cs_CV_boxes['C'].sel(param=mparam).sel(tblock=tblock).drop('tblock') * 10**6

                eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, C=C, angle_option='appenB',
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                     box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                                     verbose=True)
                eval_1D_list.append(eval_1D)
            
        eval_1D_CV_time = xr.concat(eval_1D_list, dim='time')
    
        eval_1D_CV_time.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVtime.nc')



PICOP

In [ ]:
# tuned best-estimate box parameters
tuned_box_gammas = xr.DataArray(np.array([[[0.39e-5,0.41e-5,0.44e-5,0.39e-5],
                                              [0.39e-5,0.41e-5,0.44e-5,0.39e-5]],
                                         [[0.51e-5,0.73e-5,0.92e-5,0.63e-5],
                                              [0.51e-5,0.73e-5,0.92e-5,0.63e-5]]]
                                        ), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_gammas = tuned_box_gammas.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

tuned_box_Cs = xr.DataArray(np.array([[[16.5e6,18.0e6,20.7e6,20.7e6],
                                              [16.1e6,17.8e6,20.5e6,20.5e6]],
                                    [[0.14e6,0.14e6,0.14e6,0.13e6],
                                              [0.14e6,0.14e6,0.14e6,0.13e6]]]), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_Cs = tuned_box_Cs.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

In [ ]:
# choose CV_type
CV_type = 'shelves' # can be 'shelves' or 'time'

In [ ]:
nisf_list = geometry_info_1D.Nisf

for nD_config in range(3,5):
    
    if nD_config == 3:
        pism_version = 'yes'
    elif nD_config == 4:
        pism_version = 'no'
        
    picop_opt = 'yes'
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
    
    gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version)
    C = tuned_box_Cs.sel(config=nD_config, pism=pism_version)
            
    if CV_type == 'shelves':
        
        gammas_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_gamma_CV'+CV_type+'.nc')
        E0s_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_E0_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma_plume = gammas_CV_picop['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            E0 = E0s_CV_picop['E0'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
        
            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf([kisf], T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, C=C, E0=E0, angle_option='appenB',
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                     box_tot_option='nD_config', pism_version=pism_version, picop_opt='2019', gamma_plume=gamma_plume,
                                                                     verbose=False)

            eval_1D_list.append(eval_1D)
            
        eval_1D_CV_isf = xr.concat(eval_1D_list, dim='Nisf')
            
        eval_1D_CV_isf.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        gammas_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_gamma_CV'+CV_type+'.nc')
        E0s_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_E0_CV'+CV_type+'.nc')
        
        eval_1D_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma_plume = gammas_CV_picop['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock') 
                E0 = E0s_CV_picop['E0'].sel(param=mparam).sel(tblock=tblock).drop('tblock') 

                eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, C=C, E0=E0, angle_option='appenB',
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                     box_tot_option='nD_config', pism_version=pism_version, picop_opt='2019', gamma_plume=gamma_plume,
                                                                     verbose=True)
                

                eval_1D_list.append(eval_1D)
            
        eval_1D_CV_time = xr.concat(eval_1D_list, dim='time')
    
        eval_1D_CV_time.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_CVtime.nc')



COMPUTE EVALUATION METRICS WITH ORIGINAL PARAMETERS

SIMPLE

In [ ]:
#######
favier_gammas = xr.DataArray(np.array([2.030e-5,99.32e-5,132.9e-5]),dims='param')
favier_gammas = favier_gammas.assign_coords({'param': ['linear_local', 'quadratic_local', 'quadratic_mixed_mean']})
#favier_gammas = xr.DataArray(np.array([1.060e-5,36.23e-5,36.3e-5]),dims='param')
#favier_gammas = favier_gammas.assign_coords({'param': ['linear_local', 'quadratic_local', 'quadratic_mixed_mean']})
#######
lipscomb_values_mpery = xr.DataArray(np.array([[1.11*10**4,1.44*10**4,2.06*10**6],
                                              [4.95*10**4,1.59*10**5,5.37*10**6]]), dims={'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_mean_locslope']})
lipscomb_values_mpery = lipscomb_values_mpery.assign_coords({'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_locslope']})

lipscomb_values_mpers = lipscomb_values_mpery/yearinsec

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 
#simple_param_types = ['quadratic_local','quadratic_mixed_locslope']

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in simple_param_types:
        
    for gamma_choice in ['favier','lipscomb']:
    #for gamma_choice in ['tuned']:
                
        if gamma_choice == 'favier':
            if mparam in favier_gammas.param.values:
                gamma = favier_gammas.sel(param=mparam)
                eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                    geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                    mparam, gamma, U_param=False, 
                                                                    box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D)
                
                eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_favier.nc')
                
        elif gamma_choice == 'lipscomb':
            if mparam in lipscomb_values_mpers.param.values:
                for ccalib in lipscomb_values_mpers.calib:
                    gamma = lipscomb_values_mpers.sel(param=mparam, calib=ccalib)
                    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                    geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                    mparam, gamma, U_param=False, 
                                                                    box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D) 

                    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_lipscomb_'+str(ccalib.values)+'.nc')

PLUMES

In [ ]:
plume_param_types = ['lazero19_2','lazero19_modif2']
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in plume_param_types:
    
        gamma = gamma_eff_T_lazero
        E0 = E0_lazero
                
        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                             geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                             mparam, gamma, E0=E0, 
                                                              box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)
    
        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_orig_oneFRIS.nc')

BOXES

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 


for nD_config in range(1,5):
    
    for pism_version in ['yes']: #
        
        for picop_opt in ['no']: 

            mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

            C = 1e6
            gamma = 2e-5

            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                     mparam, gamma, C=C, angle_option='appenB',
                                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                                     box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                                     verbose=True)

            eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_orig_oneFRIS.nc')

PICOP

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 


for nD_config in range(3,5):
    
    if nD_config == 4:
        pism_version = 'no'
    elif nD_config == 3:
        pism_version = 'yes'

    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picopyes'

    picop_opt='2019'
    C = C_pico
    gamma = gT_star_pico

    gamma_plume = gamma_eff_T_lazero
    E0 = E0_lazero
    
    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                          C=C, E0=E0, angle_option='appenB',
                                          box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                          pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume)
    
    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_oneFRIS_orig.nc')
